In [ ]:
import pandas as pd
import geopandas as gpd
import requests as re
from io import BytesIO
from os import path, makedirs
from unidecode import unidecode

# Produção de habitação de interesse social

Vários indicadores do GT Urbanismo são relacionados à produção e entrega de unidades de habitação de interesse social. O formulário 15 é relacionado ao orçamento utilizado na produção de unidades de habitação de interesse social. O primeiro indicador físico está no formulário 18 e diz respeito à meta de provimento de moradias do PdM 2021-2024. O formulário 20 é relacionado a uma iniciativa específica do PdM, mas os dados de execução estão disponíveis apenas no relatório da Função Habitação. Por último, os formulários 19 e 21 são relacionado, mas não possuem meta no PdM, apenas nos ODS da Agenda 2030.

Posteriormente, um novo documento apresentou várias informações diferentes relacionadas ao orçamento da função habitação, onde está incluso o Programa 3002. Portanto, vamos manter todo o orçamento da função habitação e apresentar os diferentes recortes no Qlik Sense.

## ObservaSampa

Como vamos usar vários indicadores do ObservaSampa parece fazer sentido usar uma função genérica para download de indicadores do ObservaSampa.

In [ ]:
indicadores_observa = [
    {
        "nome_completo": "11.01.02 Número de famílias beneficiadas por procedimentos de regularização fundiária em núcleos urbanos informais",
        "nome_curto": "indicador_110102",
        "id_observa": 283,
        "nome_coluna_valor": "qtd_familias"
    },

    {
        "nome_completo": "11.01.06 Número de famílias beneficiadas com obras de urbanização de assentamentos precários",
        "nome_curto": "indicador_110106",
        "id_observa": 402,
        "nome_coluna_valor": "qtd_familias"
    }, 

    {
        "nome_completo": "11.01.07 Número estimado de domicílios em favelas",
        "nome_curto": "indicador_110107",
        "id_observa": 288,
        "nome_coluna_valor": "qtd_domicilios"
    },

    {
        "nome_completo": "Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia",
        "nome_curto": "indicador_tpu",
        "id_observa": 662,
        "nome_coluna_valor": "qtd_termos"
    },
    {
        "nome_completo": "Produção de habitação de interesse social",
        "nome_curto": "indicador_producao",
        "id_observa": 285,
        "nome_coluna_valor": "qtd_unidades"
    },
    {
        "nome_completo": "01.05.02 Número de famílias em atendimento habitacional provisório (Auxílio Aluguel) por situação de risco e emergência",
        "nome_curto": "indicador_010502",
        "id_observa": 660,
        "nome_coluna_valor": "qtd_familias"
    },
    {
        "nome_completo": "11.01.08 Total de famílias beneficiárias de Auxílio Aluguel por ano",
        "nome_curto": "indicador_110108",
        "id_observa": 547,
        "nome_coluna_valor": "qtd_familias"
    },
    {
        "nome_completo": "Número de domicílios particulares não ocupados por subprefeitura",
        "nome_curto": "indicador_domicilios_nao_ocupados",
        "id_observa": None,  # defina quando houver
        "nome_coluna_valor": "domicilios_particulares_nao_ocupados"
    },
]

In [ ]:
_url_dist_mun = (
    "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:distrito_municipal&outputFormat=application/json"
)
_url_subs = (
    "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:subprefeitura&outputFormat=application/json"
)
_url_dist_censo = (
    "https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/gpkg/UF/SP/SP_distritos_CD2022.gpkg"
)

In [ ]:
def extrair_observa(id_observa:int) -> pd.DataFrame:
    url_observa = f'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador={id_observa}'
    df_observa = pd.read_csv(url_observa, sep=';', decimal=',', encoding='utf8')
    return df_observa

In [ ]:

def padronizar_subprefeitura(regiao: str) -> str:
    if pd.isna(regiao): return 'Sem informação'
    return unidecode(regiao).strip().lower()

In [ ]:
def tratar_indicador_regionalizado(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df['nivel_regional'] == 'Subprefeitura']
    return df

In [ ]:
def tratar_indicador_nao_regionalizado(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df['nivel_regional'] == 'Município']
    df = df.drop(columns=['nivel_regional', 'região'])
    return df

In [ ]:
def tratar_anos(df: pd.DataFrame,
                nome_coluna_valor:str,
                anos: list[str]=['2022', '2023', '2024']) -> pd.DataFrame:
    
    # Detectar colunas ID dinamicamente
    id_cols = [col for col in ['região', 'indicador'] if col in df.columns]
    
    df = df.melt(
        id_vars=id_cols,
        value_vars=[col for col in df.columns if col in anos],
        var_name='ano',
        value_name=nome_coluna_valor
    )
    return df

In [ ]:
def pipeline_completa(indicador: str) -> pd.DataFrame:
    indicador_info = next((item for item in indicadores_observa if item['nome_curto'] == indicador), None)
    
    df_bom = extrair_observa(indicador_info['id_observa'])

    if df_bom['nivel_regional'].str.lower().str.contains('subprefeitura').any():
        df_bom = tratar_indicador_regionalizado(df_bom)
    else:
        df_bom = tratar_indicador_nao_regionalizado(df_bom)

    df_bom = tratar_anos(df_bom, nome_coluna_valor=indicador_info['nome_coluna_valor'])

    return df_bom

### 11.01.02 Número de famílias beneficiadas por procedimentos de regularização fundiária em núcleos urbanos informais


In [ ]:

df_110102 = pipeline_completa('indicador_110102')
df_110102.head()

### 01.05.02 Número de famílias em atendimento habitacional provisório (Auxílio Aluguel) por situação de risco e emergência

In [ ]:
df_010502 = pipeline_completa('indicador_010502')
df_010502.head()

### 11.01.06 Número de famílias beneficiadas com obras de urbanização de assentamentos precários

In [ ]:
df_110106 = pipeline_completa('indicador_110106')
df_110106.head()

### 11.01.07 Número estimado de domicílios em favelas

In [ ]:
df_110107 = pipeline_completa('indicador_110107')
df_110107.head()

### 11.01.08 Total de famílias beneficiárias de Auxílio Aluguel por ano

In [ ]:
df_110108 = pipeline_completa('indicador_110108')
df_110108.head()

### Produção de habitação de interesse social

In [ ]:
df_his = pipeline_completa('indicador_producao')
df_his.head()

O indicador do formulário 19 não está ligado diretamente a nenhuma meta específica do PdM, mas está em consonância com o indicador 11.01.03 dos ODS da Agenda 2030. Os dados desse indicador estão disponíveis no [Observasampa](https://observasampa.prefeitura.sp.gov.br/).

### Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

Esse indicador possui é um exemplo da transversalidade de gênero, que integra a perspectiva de gênero na construção da politica pública voltada à provisão de habitação de interesse social. Os dados desse indicador estão disponíveis, de maneira não regionalizada, no [Observasampa](https://observasampa.prefeitura.sp.gov.br/), no indicador 05.0a.04. Posteriormente, os dados regionalizados estarão presentes no relatório da Função Habitação.

In [ ]:
df_tpu = pipeline_completa('indicador_tpu')
df_tpu.head()

### Indicador domicilios não ocupados (censo 2022)

In [ ]:
if "dist_mun" not in globals():
    _url_dist_mun = "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:distrito_municipal&outputFormat=application/json"
    dist_mun = gpd.read_file(_url_dist_mun)

if "subs" not in globals():
    _url_subs = "https://wfs.geosampa.prefeitura.sp.gov.br/geoserver/geoportal/wfs?version=1.1.0&request=GetFeature&typeName=geoportal:subprefeitura&outputFormat=application/json"
    subs = gpd.read_file(_url_subs)

if "dist_censo" not in globals():
    _url_dist_censo = "https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/gpkg/UF/SP/SP_distritos_CD2022.gpkg"
    dist_censo = gpd.read_file(_url_dist_censo)

dist_censo = dist_censo[dist_censo["CD_MUN"] == "3550308"].copy()
dist_censo["nm_distrito_municipal"] = dist_censo["NM_DIST"].apply(unidecode).str.upper()

dist_mun = dist_mun.merge(
    dist_censo[["nm_distrito_municipal", "v0003", "v0007"]],
    on="nm_distrito_municipal",
    how="left"
)

dist_mun = dist_mun.rename(columns={
    "v0003": "total_domicilios_particulares",
    "v0007": "domicilios_particulares_ocupados"
}).astype({
    "total_domicilios_particulares": "Int64",
    "domicilios_particulares_ocupados": "Int64"
})

dist_mun["domicilios_particulares_nao_ocupados"] = (
    dist_mun["total_domicilios_particulares"] - dist_mun["domicilios_particulares_ocupados"]
)

dist_mun["cd_subprefeitura"] = dist_mun["cd_identificador_subprefeitura"].apply(
    lambda cd: f"0{int(cd)}" if int(cd) < 10 else str(int(cd))
)

dom_subs = dist_mun.groupby("cd_subprefeitura", as_index=False).agg({
    "total_domicilios_particulares": "sum",
    "domicilios_particulares_ocupados": "sum",
    "domicilios_particulares_nao_ocupados": "sum"
})

subs_out = subs.merge(dom_subs, on="cd_subprefeitura", how="left")

indicador_domicilios_nao_ocupados_df = subs_out[[
    "cd_subprefeitura",
    "nm_subprefeitura",
    "domicilios_particulares_nao_ocupados",
    "total_domicilios_particulares",
    "domicilios_particulares_ocupados"
]].copy()

In [ ]:
indicador_domicilios_nao_ocupados_df

# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [ ]:
output_dir = path.join('dados', 'urbanismo')
if not path.exists(output_dir):
    makedirs(output_dir)

for nome, df in [
    ('his_entregue_original', df_his),
    ('tpu_emitido_original', df_tpu),
    ('indicador_110107', df_110107),
    ('indicador_110102', df_110102),
    ('indicador_110106', df_110106),
    ('indicador_010502', df_010502),
    ('indicador_110108', df_110108),
    ('indicador_domicilios_nao_ocupados', indicador_domicilios_nao_ocupados_df),
]:
    filename = path.join(output_dir, nome)
    df.to_csv(f'{filename}.csv', sep=';', decimal=',', encoding='utf8', index=False)
